In [1]:
import torch.nn as nn
import torch
from torch import Tensor
from typing import Type
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torch.optim as optim
import numpy as np
import random
import math
import torch.nn.functional as F
from aircraft import Aircraft

In [2]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
np.random.seed(seed)
random.seed(seed)

In [3]:
class BasicBlock(nn.Module):
    def __init__(
        self, 
        in_channels: int,
        out_channels: int,
        stride: int = 1,
        expansion: int = 1,
        downsample: nn.Module = None
    ) -> None:
        super(BasicBlock, self).__init__()
        self.expansion = expansion
        self.downsample = downsample
        self.conv1 = nn.Conv2d(
            in_channels, 
            out_channels, 
            kernel_size=3, 
            stride=stride, 
            padding=1,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            out_channels, 
            out_channels*self.expansion, 
            kernel_size=3, 
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels*self.expansion)

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return  out


In [4]:
class ResNet(nn.Module):
    def __init__(
        self, 
        img_channels: int,
        num_layers: int,
        block: Type[BasicBlock]
    ) -> None:
        super(ResNet, self).__init__()
        if num_layers == 18:
            layers = [2, 2, 2, 2]
            self.expansion = 1
        
        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            in_channels=img_channels,
            out_channels=self.in_channels,
            kernel_size=7, 
            stride=2,
            padding=3,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.final_downsample =  nn.Sequential(
                nn.Conv2d(
                    512, 
                    128,
                    kernel_size=1,
                    stride=1,
                    bias=False 
                ),
                nn.BatchNorm2d(128)
            )
        # self.fc = nn.Linear(512, 128)

    def _make_layer(
        self, 
        block: Type[BasicBlock],
        out_channels: int,
        blocks: int,
        stride: int = 1
    ) -> nn.Sequential:
        downsample = None
        if stride != 1:
            """
            This should pass from `layer2` to `layer4` or 
            when building ResNets50 and above. Section 3.3 of the paper
            Deep Residual Learning for Image Recognition
            (https://arxiv.org/pdf/1512.03385v1.pdf).
            """
            downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels, 
                    out_channels*self.expansion,
                    kernel_size=1,
                    stride=stride,
                    bias=False 
                ),
                nn.BatchNorm2d(out_channels * self.expansion),
            )
        layers = []
        layers.append(
            block(
                self.in_channels, out_channels, stride, self.expansion, downsample
            )
        )
        self.in_channels = out_channels * self.expansion

        for i in range(1, blocks):
            layers.append(block(
                self.in_channels,
                out_channels,
                expansion=self.expansion
            ))
        return nn.Sequential(*layers)

    def forward(self, x: Tensor) -> Tensor:
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.final_downsample(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return x

In [5]:
class KernelNN(nn.Module):
    def __init__(self):
        super(KernelNN, self).__init__()
        self.fc1 = nn.Linear(256, 128)
        self.bn = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn(x)
        x = F.relu(self.fc2(x))
        x = self.bn(x)
        x = torch.sigmoid(self.fc3(x))
        return x

In [6]:
class Net_NW_Head(nn.Module):

    def __init__(self, classes, device):
        self.device = device
        super(Net_NW_Head, self).__init__()
        self.rs18 = ResNet(img_channels=3, num_layers=18, block=BasicBlock)
        self.kernel_nn = KernelNN()
        self.classes = classes
        self.eps = 1e-5
        
    def set_support(self, support_element_imgs, support_labels):
        self.support_element_imgs = support_element_imgs
        self.support_labels_one_hot = torch.tensor(np.eye(self.classes)[support_labels], requires_grad=True).to(device)

    def apply_kernel(self, x):
        x_and_sup_concatenated = []
        for x_i in x:
            for s_i in self.support_elements:
                x_and_sup_concatenated.append(torch.cat([x_i, s_i]).unsqueeze(0))
        x_and_sup_concatenated = torch.cat(x_and_sup_concatenated)

        raw_distances = self.kernel_nn(x_and_sup_concatenated)
        raw_distances = torch.reshape(raw_distances, (len(x), len(self.support_elements)))

        raw_distances = raw_distances + self.eps
      
        distances = raw_distances / raw_distances.sum(dim=-1).unsqueeze(-1)
        result = []
        for i in range(distances.shape[0]):
            result.append((distances[i].unsqueeze(1) * self.support_labels_one_hot).sum(dim = 0).unsqueeze(0))
        result = torch.cat(result)
        return result

    def forward(self, x):

        x = self.rs18(x)
        self.support_elements = self.rs18(self.support_element_imgs)
        x = self.apply_kernel(x)
        x = x + self.eps
        x = torch.log(x)
        return x


In [7]:
plt.style.use('ggplot')
IMG_SIZE = 224

def get_data(batch_size=64):

    dataset_train = Aircraft('./data',
        train=True,
        download=True,
        transform=transforms.Compose([
              transforms.Resize((IMG_SIZE,IMG_SIZE)),
              transforms.RandomHorizontalFlip(p=0.5),
              transforms.RandomVerticalFlip(p=0.3),
              transforms.RandomRotation(degrees=(-15, 15)),
              transforms.ToTensor()
           ])
    )
    dataset_valid = Aircraft('./data',
         train=False,
         download=True,
         transform=transforms.Compose([
              transforms.Resize((IMG_SIZE,IMG_SIZE)),
              transforms.ToTensor()
           ])
    )
    train_loader = DataLoader(
        dataset_train, 
        batch_size=batch_size,
        shuffle=True
    )
    valid_loader = DataLoader(
        dataset_valid, 
        batch_size=batch_size,
        shuffle=False
    )
    return train_loader, valid_loader, dataset_train, dataset_valid
  

In [8]:
def save_plots(train_acc, valid_acc, train_loss, valid_loss, name=None):
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_acc, color='tab:blue', linestyle='-', 
        label='train accuracy'
    )
    plt.plot(
        valid_acc, color='tab:red', linestyle='-', 
        label='validataion accuracy'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(os.path.join('outputs', name+'_accuracy.png'))
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_loss, color='tab:blue', linestyle='-', 
        label='train loss'
    )
    plt.plot(
        valid_loss, color='tab:red', linestyle='-', 
        label='validataion loss'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join('outputs', name+'_loss.png'))

In [9]:
SUPPORT_SIZE = 3

def train(model, trainloader, optimizer, criterion, device, full_support_set):
    model.train()
    print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):

        support_elements_temp = []
        support_labels = []
        for label in full_support_set.keys():
            support_elements_temp.extend(random.sample(full_support_set[label], SUPPORT_SIZE))
            support_labels.extend([label for i in range(SUPPORT_SIZE)])

        img_shape = support_elements_temp[0].shape
        support_elements_ng = torch.zeros((len(support_elements_temp), img_shape[0], img_shape[1], img_shape[2]))
        for i in range(len(support_elements_temp)):
          support_elements_ng[i] = support_elements_temp[i]

        support_elements = support_elements_ng.clone().detach().requires_grad_(True)
        support_elements = support_elements.to(device)

        counter += 1
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        # support
        model.set_support(support_elements, support_labels)

        outputs = model(image)

        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()
        loss.backward()
        optimizer.step()
    
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc

def validate(model, testloader, criterion, device, full_support_set):
    model.eval()
    print('Validation')
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0

    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader), total=len(testloader)):

            support_elements_temp = []
            support_labels = []
            for label in full_support_set.keys():
                support_elements_temp.extend(random.sample(full_support_set[label], SUPPORT_SIZE))
                support_labels.extend([label for i in range(SUPPORT_SIZE)])

            img_shape = support_elements_temp[0].shape
            support_elements = torch.zeros((len(support_elements_temp), img_shape[0], img_shape[1], img_shape[2]))
            for i in range(len(support_elements_temp)):
                support_elements[i] = support_elements_temp[i]

            support_elements = support_elements.to(device)

            counter += 1
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)

            # support
            model.set_support(support_elements, support_labels)
            outputs = model(image)
            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()
        
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(testloader.dataset))
    return epoch_loss, epoch_acc

In [10]:


epochs = 350
batch_size = 16
learning_rate = 0.001
momentum = 0.9
classes = 100

device = torch.device("cuda:4" if torch.cuda.is_available() else 'cpu')


In [11]:
train_loader, valid_loader, dataset_train, dataset_valid = get_data(batch_size=batch_size)

In [12]:
len(dataset_valid)

3333

In [13]:
full_support_set = dict()
for data, i in dataset_train:
    full_support_set.setdefault(i, []).append(data)

In [14]:
device

device(type='cuda', index=4)

In [15]:
model = Net_NW_Head(classes, device).to(device)
plot_name = 'nw_scratch'

In [16]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
criterion = F.nll_loss

In [17]:
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []

for epoch in range(epochs):
    print(f"[INFO]: Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_acc = train(
        model, 
        train_loader, 
        optimizer, 
        criterion,
        device,
        full_support_set
    )
    valid_epoch_loss, valid_epoch_acc = validate(
        model, 
        valid_loader, 
        criterion,
        device,
        full_support_set
    )
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    print(learning_rate)
    torch.save(model.state_dict(), 'test_nw_fc_FGVCA2_w')
            
    print(f"Training loss: {train_epoch_loss:.3f}, training acc: {train_epoch_acc:.3f}")
    print(f"Validation loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f}")
    print('-'*50)

[INFO]: Epoch 1 of 350
Training


100%|██████████| 417/417 [08:13<00:00,  1.18s/it]


Validation


100%|██████████| 209/209 [02:11<00:00,  1.59it/s]


0.001
Training loss: 4.533, training acc: 2.220
Validation loss: 4.500, validation acc: 2.670
--------------------------------------------------
[INFO]: Epoch 2 of 350
Training


...

Validation


100%|██████████| 209/209 [02:15<00:00,  1.54it/s]


0.001
Training loss: 0.830, training acc: 73.556
Validation loss: 1.541, validation acc: 59.826
--------------------------------------------------
[INFO]: Epoch 77 of 350
Training


 12%|█▏        | 51/417 [01:02<07:27,  1.22s/it]


KeyboardInterrupt: 

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

for epoch in range(34, epochs):
    print(f"[INFO]: Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_acc = train(
        model, 
        train_loader, 
        optimizer, 
        criterion,
        device,
        full_support_set
    )
    valid_epoch_loss, valid_epoch_acc = validate(
        model, 
        valid_loader, 
        criterion,
        device,
        full_support_set
    )
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    print(learning_rate)
    torch.save(model.state_dict(), 'test_nw_fc_FGVCA2_w')
            
    print(f"Training loss: {train_epoch_loss:.3f}, training acc: {train_epoch_acc:.3f}")
    print(f"Validation loss: {valid_epoch_loss:.3f}, validation acc: {valid_epoch_acc:.3f}")
    print('-'*50)

[INFO]: Epoch 35 of 350
Training


 37%|███▋      | 156/417 [03:13<05:19,  1.22s/it]